# Cvičenie 7: Monte Carlo simulácie

Výpočtové modely a simulácie sú dôležitým nástrojom pri zodpovedaní otázok, ktorých riešenie vyžaduje dlhé matematické odvádzačky a komplikované definície rôznych javov. V niektorých prípadoch chceme skúmať systém, ktorý je natoľko komplikovaný, že presahuje možnosti ľudského chápania. Inokedy systém nemôžeme popísať iba matematickými modelmi, alebo potrebujeme zaviesť niekoľko zjednodušení, aby sme vôbec dokázali aproximovať jeho fungovanie. Vo všetkých týchto scenároch nám pomôžu simulácie, z ktorých sa dnes zameriame na Monte Carlo simulácie.

Monte Carlo simulácie slúžia na popis komplikovaných javov tak, že namiesto presných pozorovaní nasimulujeme dostatočne veľký počet inštancií skúmaného javu na to, aby sme si dokázali z výsledkov urobiť dôležité závery. Tento princíp určite poznáte z prieskumov verejnej mienky, kde namiesto toho, aby sa zamestnanci agentúr opýtali každého z nás na náš názor, urobia si prehľad názorov na menšej reprezentatívnej vzorke respondentov, a dosiahnuté výsledky namapujú na celú populáciu. Rovnaký prístup nám pomôže aj pri odpovedaní niekoľko ťažších matematických otázok, alebo aj pri aproximácii hodnoty $\pi$.

**Poznámka:** Vzhľadom na to, že v simuláciách často budeme modelovať neisté javy cez pravdepodobnosť, [naštudujte si možnosti, ktoré ponúka modul `random`](https://docs.python.org/3/library/random.html).

## Čo to bude? Auto alebo koza?

Možno ste už počuli o pravdepodobnostnej hádanke, ktorá je známa ako problém Montyho Halla. Predstavte si, že ste v televíznej relácii, kde hráte o hlavnú cenu - o úplne nové auto. Producenti relácie umiestnili auto za jedny z troch dverí. Za každými z ostávajúcich dverí je cena útechy – koza. Vašou úlohou je vybrať si jedny dvere. Následne moderátor otvorí jedny z dvoch zostávajúcich dverí, ale iba tie, za ktorými je koza. Teraz príde ale moment veľkej dilemy: máte možnosť buď ponechať svoj pôvodný výber, alebo zmeniť voľbu na zostávajúce dvere. V oboch prípadoch vyhrávate cenu, ktorá je za dverami, ktoré ste si zvolili, a samozrejme najradšej by ste išli domov s autom. 

Na začiatku hry ste si napríklad zvolili dvere číslo 1. Moderátor následne otvorí dvere číslo 3, za ktorými je koza. Čo si myslíte, zvýši sa vaša šanca na výhru auta, ak zmeníte voľbu na dvere číslo 2, alebo mali by ste si ponechať pôvodný výber? Čo je správnym krokom?

Keďže táto dilema má kontraintuitívne riešenie a zmiatla prekvapujúco veľký počet ľudí (medzi nimi aj matematikov), namiesto toho aby sme sa pustili do komplikovanej analýzy pravdepodobnosti vytvoríme jednoduchú implementáciu simulácie hry a porovnáme výhernosť pri stratégii, kde si našu voľbu zmeníme, a keď ostaneme pri pôvodne vybratých dverách.

Najprv si implementujte funkciu `simulate_one_game()`, ktorá má jeden parameter `switch` vyjadrujúc, či si simulovaný súťažiaci zmení svoju voľbu (`True`) alebo nie (`False`). Vo funkcii si náhodne vygenerujte dvere, za ktorými bude hlavná výhra, následne náhodne vygenerujte výber hráča. Potom z možných volieb odstráňte jedny dvere, za ktorými nie je hlavná výhra a potom upravte voľbu hráča (ak je to potrebné). Na konci funkcia nech vráti hodnotu `True` alebo `False` podľa toho, či hráč vyhral (jeho výber na konci je totožný ako číslo dverí, za ktorými je hlavná cena).

In [ ]:
import random


def simulate_one_game(switch):
    # set up the game, put the prize behind one door
    
    # initial guess - can be any
    
    # open one door: it cannot be the player's guess
    # nor the door behind which lies the prize
    
    # the might player changes his decision, takes the other available door

    # the player wins if his final guess is the same as the door
    # behind which lies the prize
    return False

Asi je nám všetkým jasné, že jedna nasimulovaná hra nám nedá presný odhad skutočnej pravdepodobnosti výhry pri istej stratégii. Práve preto máme implementovanú funkciu `check_behavior()`, ktorá vykoná istý počet pokusov a vráti celkovú vypočítanú pravdepodobnosť. Následne vo funkcii `find_answer()` otestujeme obe možné stratégie a uvidíme výhernosť pri rôznych rozhodnutiach hráča. Prekvapuje vás výsledok? Vyskúšajte rôzne počty testovacích prípadov pri volaní funkcie `check_behavior()` a skúste nájsť hranicu, kde sa už vypočítaná pravdepodobnosť príliš nemení.

In [ ]:
def check_behavior(switch, test_count):
    wins = 0

    for _ in range(test_count):
        wins += simulate_one_game(switch)

    return wins / test_count


def find_answer():
    test_count = 1000000
    print("Win percentage when switching: {:.2%}".format(
        check_behavior(True, test_count)))
    print("Win percentage when not switching: {:.2%}".format(
        check_behavior(False, test_count)))

find_answer()

## Všetko najlepšie k narodeninám! Ale koľkým?

Stalo sa vám, že v kruhu vašich známych a kamarátov ste našli dvoch, ktorí majú narodeniny v rovnaký deň? Vzhľadom na nie úplnú uniformnú distribúciu narodenín v populácii (mierna prevaha septembrových, najmenej ľudí sa narodí vo februári) je táto skutočnosť na prvý pohľad neprekvapujúca. Na druhej strane ale jedinečnosť narodenín v skupine ľudí by bola veľmi nepravdepodobná aj keby sme mali úplne rovnaké zastúpenie všetkých dní v celkovej populácii ľudí.

Vzhľadom na to, že v roku máme 365 dní (keď sa niekto narodí 29. 2., z administratívnych dôvodov má pridelené narodeniny buď 28. 2. alebo 1. 3.), je jasné, že na to, aby sme mali 100 % pravdepodobnosť, že aspoň dvaja v skupine majú v rovnaký deň narodeniny potrebujeme 366 ľudí. Ale pri akom počte ľudí bude pravdepodobnosť zdieľaných narodenín väčšia ako 50 %? Väčšia ako 90 %? Väčšia ako 99 %? Odpoveď môže byť znovu prekvapujúca.

Ako prvé implementujeme funkciu `generate_birthdays()`, ktorá podľa parametra `n` vygeneruje náhodné narodeniny daného počtu ľudí. Pre každého vygenerujte dátum narodenia nezávisle z intervalu 1 až 365 (deň v roku, nerátame s priestupným rokom), funkcia teda vracia zoznam *n* celých čísel.

Ďalej implementujte funkciu `has_match()`, ktorá z vygenerovaného zoznamu zistí, či existujú aspoň dvaja ľudia, ktorí oslavujú svoje narodeniny v rovnaký deň. Funkcia na základe toho vracia hodnotu `True` alebo `False`.

In [ ]:
import random


def generate_birthdays(n):
    return []


def has_match(birthdays):
    return False

V ďalšom kroku implementujeme funkciu `test_prob()`, ktorá vykoná niekoľko pokusov s rovnakou veľkosťou nasimulovaných skupín ľudí (`n`), aby sme mohli získať čo najpresnejšiu aproximáciu pravdepodobnosti. Presnosť vieme nastavovať cez parameter `test_count` - čím viac testov, tým presnejší výsledok. Funkcia vracia pravdepodobnosť zdieľaných narodenín pri istej veľkosti skupiny. Ako vždy pri pravdepodobnosti, potrebujete predeliť počet výhodných výstupov (prípady, kde niekoľko ľudí má narodeniny v rovnaký deň) s celkovým počtom pokusov (`test_count`).

Následne môžete otestovať niekoľko nastavení, a nájsť odpoveď na otázku, koľko ľudí potrebujete, aby ste mali viac ako 50 % šancu na zdieľané narodeniny.

In [ ]:
def test_prob(n, test_count):
    return 0.0


n = 10
test_count = 10000
print("Probability of shared birthday with {} people: {:.6%}".format(
      n, test_prob(n, test_count)))


## Krátka história $\pi$


Jeden z najznámejších matematických konštánt je hodnota $\pi$ (alebo Ludolfovo číslo aka Archimedova konštanta), ktorá vyjadruje pomer obvodu kruhu k jeho priemeru. Síce samotný tvorca Pythonu preferuje konštantu $\tau$, ktorá vyjadruje pomer obvodu kruhu k jeho polomeru ([viac o debate tu](https://tauday.com/tau-manifesto)), na dnešnom cvičení sa budeme zaoberať práve $\pi$, prečo má práve hodnotu, akú má, a ako vôvec túto hodnotu poznáme?

Konštantu $\pi$ poznáte zo školy, kde ste pravdepodobne použili aproximáciu jej hodnoty $3.14$. Táto aproximácia nám stačí vo väčšine prípadov, samotná konštanta je ale irracionálne číslo, ktoré nevieme vyjadriť ako podiel dvoch celých čísel. Čo je ešte horšie, hodnotu nevieme vyjadriť ani ako číslo s periodicky sa opakujúcou desatinnou časťou (aspoň podľa nášho dnešného poznania).

Pomer obvodu a priemeru kruhu zaujímal vedcov už aj v staroveku, konštantu poznali už niekedy 2000 rokov pred n.l. Prvý postup pre jej výpočet ale objavil Archimedes. K riešeniu sa dopracoval aproximáciou kruhu pomocou vpísaného mnohouholníka (konkrétne použil 96 uholník), ktorého obvod vieme veľmi jednoducho vypočítať a slúží ako dobrý odhad obvodu kruhu. Takýmto spôsobom Archimedes zistil, že hodnota je medzi $\frac{223}{71}$ a $\frac{220}{70}$, ($3,1408 < \pi < 3,1428$). Vizualizáciu postupu vidíte na obrázku nižšie

![Výpočet pi pomocou vpísaných mnohouholníkov](https://www.craig-wood.com/nick/pub/pymath/pi_geometric_inscribed_polygons.png)

Túto myšlienku bral vážne aj Ludolph van Ceulen, ktorý konštantu upresnil na 35 miest - potreboval k tomu pravidelný mnohouholník s 1 073 741 284 stranami a kopu voľného času. Označenie $\pi$ presadil Leonhard Euler, ktorý objavil aj [vlastnú užitočnú konštantu](https://sk.wikipedia.org/wiki/Eulerovo_číslo).

Moderné výpočty hodnoty $\pi$ samozrejme používajú aj počítače, v roku 1949 počítač ENIAC vypočítal hodnotu na 2037 desatinných miest za 70 hodín. Dnes poznáme hodnotu prvých 31 biliónov miest.

## Výpočet hodnoty $\pi$

Presnú hodnotu $\pi$ vieme vyjadriť ako nekonečný rad

$\pi = 4 \sum_{k=0}^{\infty} \frac{(-1)^k}{2k + 1} = \frac{4}{1} - \frac{4}{3} + \frac{4}{5} - \frac{4}{7} + \frac{4}{9} - \frac{4}{11} + ...$

Existuje ale aj iná aproximácia, ktorú používali niektoré počítačové programy pre výpočet tejto hodnoty. Konkrétne využijeme známy vzorec pre výpočet obsahu kruhu: $S = \pi r^2$. Ako pomôcku zoberieme štvorec s vpísanou štvrťou kruhu, ako môžete vidieť na obrázku:

<img src="sources/lab07/estimation.png" width="400">

V tomto príklade obsah štvorca je $S_s = r^2$ keďže dĺžka strany je rovná polomeru kruhu. Obsah jednej štvrti kruhu je zase $S_c=\frac{\pi r^2}{4}$. Hodnotu $\pi$ vieme vyjadriť aj ako pomer týchto dvoch oblastí (pomer zafarbenej časti a celého štvorca):

$\frac{S_c}{S_s} = \frac{\frac{\pi r^2}{4}}{r^2} = \frac{\pi r^2}{4 r^2} = \frac{\pi}{4}$

Z toho vieme, že:

$\pi = 4 \frac{S_c}{S_s}$

Otázkou ostáva, ako vieme aproximovať obsahy týchto dvoch útvarov? Táto metóda je založená na simuláciach typu Monte Carlo, pre náš prípad to znamená, že náhodne vygenerujeme veľký počet bodov vo štvorci, a spočítame z nich počet tých bodov, ktoré sa nachádzajú v zafarbenej časti. Ak vygenerujeme dostatočný počet bodov, tak pomer počtu bodov v zafarbenej časti a celkového počtu bodov bude dobrou aproximáciou pomeru obsahu dvoch útvarov.

V riešení zachováme pôvodnú terminológiu, kde aproximácia bola definovaná ako hod šípkami.

Na začiatku riešenia nájdete všetky potrebné importy.

In [ ]:
import matplotlib.pyplot as plt
import random
import math
from numpy import arange

## 1. Výpočet $\pi$

Implementáciu začneme funkciou `calculate_pi`, ktorá vráti odhad hodnoty tejto konštanty na základe vstupných parametrov, kde `in_circle` vyjadruje počet bodov v zafarbenej časti, a `total` je celkový počet generovaných bodov. Funkcia má jednu návratovú hodnotu - desatinné číslo, ktoré vyjadruje odhad hodnoty $\pi$.

In [ ]:
def calculate_pi(in_circle, total):
    return 0

## 2. Určenie počtu bodov v kruhu

K aproximácii hodnoty $\pi$ potrebujeme poznať aj počet bodov, ktoré sa nachádzajú v zafarbenej časti (v kruhu). Implementujte funkciu `is_in_circle`, ktorá vráti hodnotu `True` alebo `False` na základe toho, či bod na pozícii definovanej vstupnými parametrami `x` a `y` sa nachádza v kruhu. Pre jednoduchosť predpokladáme, že $r = 1$ a stred kruhu je na pozícii $[0, 0]$.

In [ ]:
def is_in_circle(x, y):
    return False

## 3. Celková simulácia

V ďalšom kroku vytvoríme celú simuláciu, pričom ju implementujeme vo funkcii `throw_darts`. Funkcia vygeneruje náhodné body (ich počet je daný parametrom `num_darts`), a vráti odhadovanú hodnotu konštanty $\pi$ pre danú simuláciu. Aby sme videli priebeh konvergencie, odhadované hodnoty ale budeme zaznamenávať po každom vygenerovanom bode. Práve preto funkcia už obsahuje dve pomocné premenné:

* `in_circle` vyjadruje počet bodov, ktoré sa nachádzajú v zafarbenej oblasti;
* `estimates` je zoznam, do ktorého pridáme odhadované hodnoty po jednotlivých iteráciach.

Funkcia iteratívne vygeneruje `num_darts` bodov, pričom pre každý bod zistí, či daný bod je v kruhu alebo nie. Zároveň vypočíta odhadovanú hodnotu $\pi$ a uloží ju do zoznamu `estimates`. Na konci vykonávania vráti finálny odhad. Keďže vykonávanie funkcie môže trvať dlhšie pri väčšom počte generovaných bodov, medzivýsledky vypisujte po 10000 iteráciach. Druhý parameter `show_plot` zatiaľ nepoužijeme.

In [ ]:
def throw_darts(num_darts, show_plot):
    in_circle = 0
    estimates = list()
    
    return 0

## 4. Vizualizácia konvergencie

V poslednom kroku vytvoríme funkciu `plot_estimates`, ktorá nám zobrazí konvergenciu odhadu k skutočnej hodnote $\pi$. Funkcia berie jeden parameter `estimates`, ktorý je zoznam odhadov po jednotlivých iteráciach z funkcie `throw_darts`. Pre vizualizáciu použite knižnicu `matplotlib`, v grafe vizualizujte priebeh odhadu, aj skutočnú hodnotu $\pi$, napríklad:

![](sources/lab07/graph.png)

Následne upravte funkciu `throw_darts`, aby automaticky zobrazila graf, ak hodnota `show_plot` bude `True`. Vo funkcii upravte aj výpočet medzivýsledkov podľa tohto parametra, odhad vypočítajte v každej iterácii len v prípade, že chceme vykresliť graf (výpočet po 10000 iteráciach ale ponechajte).

In [ ]:
def plot_estimates(estimates):
    pass

## 5. Testovanie riešenia

Vaše riešenie môžete otestovať pomocou hlavnej funkcie:

In [ ]:
darts = 1000000
pi_guess = throw_darts(darts, True)
print("Estimated value of pi with {} darts is {}".format(darts, pi_guess))